# Tutorial 22: Reflection Pattern

In this tutorial, you'll learn how to build self-improving agents that iteratively refine their outputs through reflection and critique.

**What you'll learn:**
- **Generate-Critique-Revise loop**: Iterative improvement pattern
- **Quality control**: Using critique to guide revisions
- **Approval signals**: Explicit stopping conditions
- **Multi-criteria reflection**: Structured scoring across dimensions
- **Multi-model reflection**: Using different LLMs for generation vs critique

By the end, you'll have agents that write, critique, and revise their own work to produce high-quality outputs.

## What is Reflection?

Reflection is a pattern where an LLM:
1. **Generates** an initial draft
2. **Critiques** the draft (can be same or different LLM)
3. **Revises** based on critique feedback
4. **Repeats** until approved or max iterations reached

This mirrors how humans improve their work through drafts and revisions.

### Architecture

```
START → Generator → Critic → Should Continue?
           ↑                        |
           |                        |
           └────── If Needs ────────┘
                   Revision
                                    |
                                    ├──→ END (if APPROVED)
                                    └──→ END (if max iterations)
```

### Use Cases

- **Writing**: Essays, reports, documentation, emails
- **Code Generation**: Generate, review, refactor
- **Analysis**: Initial assessment, critique, refinement
- **Creative Content**: Iterative improvement of creative outputs

## Setup

In [ ]:
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0.7,  # Some creativity for writing
)

print("LLM configured")

## Part 1: Basic Reflection Pattern

Let's start with the basic reflection pattern using our module.

In [ ]:
from langgraph_ollama_local.patterns.reflection import (
    create_reflection_graph,
    run_reflection_task,
)

# Create a basic reflection graph
graph = create_reflection_graph(
    llm=llm,
    max_iterations=3,
)

print("Reflection graph created!")

### Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not render graph: {e}")

### Example 1: Essay Writing

Let's use reflection to write a short essay with iterative improvement.

In [ ]:
task = """Write a brief essay (200-300 words) explaining why LangGraph is useful 
for building AI agents. Include specific benefits and use cases."""

print("Task:", task)
print("="*60)

result = run_reflection_task(
    graph=graph,
    task=task,
    max_iterations=3,
)

print("\n" + "="*60)
print("FINAL DRAFT:")
print("="*60)
print(result["draft"])
print("\n" + "="*60)
print(f"Iterations: {result['iteration']}")
print(f"Final Critique: {result['critique'][:100]}...")

### Example 2: Professional Email

Reflection is excellent for writing professional communications.

In [ ]:
email_task = """Write a professional email to a client explaining a 2-week delay 
in project delivery due to unexpected technical challenges. The email should:
- Acknowledge the delay professionally
- Explain the challenges briefly
- Provide a new realistic timeline
- Express commitment to quality
- Maintain a positive, confident tone"""

print("Task: Professional Email")
print("="*60)

result = run_reflection_task(
    graph=graph,
    task=email_task,
    max_iterations=3,
)

print("\n" + "="*60)
print("FINAL EMAIL:")
print("="*60)
print(result["draft"])

## Part 2: Step-by-Step Implementation

Let's build a reflection system from scratch to understand how it works.

### Step 1: Define State

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class ReflectionState(TypedDict):
    """State for reflection loop."""
    messages: Annotated[list, add_messages]  # Conversation history
    task: str                                 # Original task
    draft: str                                # Current draft
    critique: str                             # Latest critique
    iteration: int                            # Current iteration
    max_iterations: int                       # Max iterations allowed

print("State defined with: messages, task, draft, critique, iteration, max_iterations")

### Step 2: Create Generator Node

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

GENERATOR_PROMPT = """You are a skilled writer. Your task is to write or revise content.

If this is the first draft, write a complete response.
If you have received critique, revise your draft to address the feedback.

Focus on:
- Clarity and conciseness
- Accuracy and completeness
- Engaging and professional tone"""

def generate_node(state: ReflectionState) -> dict:
    """Generate or revise the draft."""
    iteration = state.get("iteration", 0)
    task = state["task"]
    draft = state.get("draft", "")
    critique = state.get("critique", "")
    
    if iteration == 0:
        # First draft
        user_msg = f"Write a response to this task:\n\n{task}"
    else:
        # Revision based on critique
        user_msg = f"""Revise this draft based on the critique.

ORIGINAL TASK: {task}

CURRENT DRAFT:
{draft}

CRITIQUE:
{critique}

Please provide an improved version:"""
    
    messages = [
        SystemMessage(content=GENERATOR_PROMPT),
        HumanMessage(content=user_msg)
    ]
    
    response = llm.invoke(messages)
    new_draft = response.content
    
    print(f"\n=== Generation (iteration {iteration + 1}) ===")
    print(new_draft[:200] + "..." if len(new_draft) > 200 else new_draft)
    
    return {
        "draft": new_draft,
        "iteration": iteration + 1,
        "messages": [AIMessage(content=f"Draft {iteration + 1} generated")]
    }

print("Generator node created")

### Step 3: Create Critic Node

In [ ]:
CRITIC_PROMPT = """You are a thoughtful editor. Review the draft and provide constructive feedback.

Evaluate:
1. Does it fully address the task?
2. Is it clear and well-organized?
3. Are there any errors or areas for improvement?
4. Is the tone appropriate?

If the draft is excellent and needs no changes, respond with exactly: "APPROVED"
Otherwise, provide specific, actionable feedback for improvement."""

def critique_node(state: ReflectionState) -> dict:
    """Critique the current draft."""
    task = state["task"]
    draft = state["draft"]
    iteration = state["iteration"]
    
    user_msg = f"""Review this draft for the given task.

TASK: {task}

DRAFT:
{draft}

Provide your critique or "APPROVED" if excellent:"""
    
    messages = [
        SystemMessage(content=CRITIC_PROMPT),
        HumanMessage(content=user_msg)
    ]
    
    response = llm.invoke(messages)
    critique = response.content
    
    print(f"\n=== Critique (iteration {iteration}) ===")
    print(critique[:200] + "..." if len(critique) > 200 else critique)
    
    return {
        "critique": critique,
        "messages": [AIMessage(content=f"Critique {iteration} provided")]
    }

print("Critic node created")

### Step 4: Create Routing Logic

In [ ]:
from langgraph.graph import END

def should_continue(state: ReflectionState) -> str:
    """Decide whether to continue refining or finish."""
    iteration = state["iteration"]
    max_iterations = state["max_iterations"]
    critique = state.get("critique", "")
    
    # Stop if approved
    if "APPROVED" in critique.upper():
        print("\n✓ Draft approved!")
        return END
    
    # Stop if max iterations reached
    if iteration >= max_iterations:
        print(f"\n✓ Max iterations ({max_iterations}) reached")
        return END
    
    print(f"\n→ Continuing to iteration {iteration + 1}")
    return "generator"

print("Routing logic created")

### Step 5: Build the Graph

In [ ]:
from langgraph.graph import StateGraph, START

# Build the reflection graph
workflow = StateGraph(ReflectionState)

# Add nodes
workflow.add_node("generator", generate_node)
workflow.add_node("critic", critique_node)

# Add edges
workflow.add_edge(START, "generator")       # Start by generating
workflow.add_edge("generator", "critic")    # Then critique
workflow.add_conditional_edges(
    "critic",
    should_continue,
    {"generator": "generator", END: END}
)

# Compile
custom_graph = workflow.compile()

print("Custom reflection graph compiled!")

### Step 6: Test the Custom Graph

In [ ]:
# Test with a technical explanation task
task = "Explain how neural networks learn through backpropagation in 150 words."

print(f"Task: {task}")
print("="*60)

result = custom_graph.invoke({
    "task": task,
    "messages": [],
    "draft": "",
    "critique": "",
    "iteration": 0,
    "max_iterations": 3,
})

print("\n" + "="*60)
print("FINAL DRAFT:")
print("="*60)
print(result["draft"])

## Part 3: Multi-Criteria Reflection

Use structured scoring across multiple criteria for more nuanced feedback.

In [ ]:
# Create multi-criteria reflection graph
multi_criteria_graph = create_reflection_graph(
    llm=llm,
    max_iterations=4,
    use_multi_criteria=True,
    approval_threshold=8,  # Require score of 8/10 or higher
)

print("Multi-criteria reflection graph created!")

In [ ]:
# Test with a complex writing task
task = """Write a technical blog post introduction (200 words) about the benefits 
of using LangGraph for AI agent development. The introduction should:
- Hook the reader with a compelling opening
- Clearly state the problem being solved
- Preview the main benefits to be discussed
- Use a professional but accessible tone"""

print("Task: Technical Blog Post Introduction")
print("="*60)

result = run_reflection_task(
    graph=multi_criteria_graph,
    task=task,
    max_iterations=4,
)

print("\n" + "="*60)
print("FINAL INTRODUCTION:")
print("="*60)
print(result["draft"])
print("\n" + "="*60)
print("FINAL CRITIQUE WITH SCORES:")
print("="*60)
print(result["critique"])

## Part 4: Multi-Model Reflection (Advanced)

Use different models for generation and critique:
- Fast model for generation (e.g., llama3.2:3b)
- Stronger model for critique (e.g., llama3.1:70b)

This can improve critique quality while keeping generation efficient.

In [ ]:
from langgraph_ollama_local.patterns.reflection import create_multi_model_reflection_graph

# Use the same model for both in this example
# In production, you might use:
# generator_llm = ChatOllama(model="llama3.2:3b")  # Fast for drafts
# critic_llm = ChatOllama(model="llama3.1:70b")     # Thorough for critique

generator_llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0.8,  # More creative for generation
)

critic_llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0.3,  # More focused for critique
)

multi_model_graph = create_multi_model_reflection_graph(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    max_iterations=3,
)

print("Multi-model reflection graph created!")

In [ ]:
# Test multi-model reflection
task = """Write a creative product description (150 words) for a smart home device 
that uses AI to learn your daily routines and automatically adjust lighting, 
temperature, and music preferences."""

print("Task: Creative Product Description")
print("="*60)

result = run_reflection_task(
    graph=multi_model_graph,
    task=task,
    max_iterations=3,
)

print("\n" + "="*60)
print("FINAL PRODUCT DESCRIPTION:")
print("="*60)
print(result["draft"])

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **Generator** | Creates initial drafts and revisions |
| **Critic** | Evaluates drafts and provides feedback |
| **Approval Signal** | Explicit "APPROVED" signal to stop refining |
| **Iteration Limit** | Safety mechanism to prevent infinite loops |
| **Multi-Criteria** | Structured scoring across multiple dimensions |
| **Multi-Model** | Different LLMs for generation vs critique |

## Reflection vs Reflexion

| Pattern | Focus | Memory | Use Case |
|---------|-------|--------|----------|
| **Reflection** | Single output improvement | Current draft only | Writing, content creation |
| **Reflexion** | Learning across attempts | Episodic memory | Complex problem-solving |

## Best Practices

1. **Set reasonable iteration limits** (3-5 typically sufficient)
2. **Use clear approval signals** ("APPROVED" in critique)
3. **Provide specific critique prompts** for better feedback
4. **Consider multi-model** for quality vs speed tradeoffs
5. **Use multi-criteria** for complex quality requirements

## What's Next?

In **Tutorial 23: Reflexion Pattern**, you'll learn:
- Learning from multiple attempts
- Episodic memory across trials
- Tool-augmented reflection
- When to use Reflexion vs Reflection